In [1]:
# -*- config : utf-8 -*-
import numpy as np
import sys
import time as ti
import warnings
from tensorflow.lite.python.interpreter import Interpreter

# 경고 메시지 무시
warnings.filterwarnings(action='ignore')

In [2]:
# --- 분석 수행 및 결과 저장 클래스 선언 ---
class EsNoiseAnalyzer:
    
    def __init__(self, model_path):
        self.model_path = model_path
        self.elapsed_time = 0
        self.predict_list = []
        
    def analysis(self, input_list):
        print("START : " + self.model_path)
            
        # 모델 로딩
        model_content = None
        with open(self.model_path, 'rb') as f:
            model_content = f.read()
            
        model_content = bytes(model_content)
            
        interpreter = Interpreter(model_content = model_content)
        interpreter.allocate_tensors()
            
        input_details = interpreter.get_input_details()
        input_index = input_details[0]['index']
        input_scale, input_zero_point = input_details[0]['quantization']
            
        print(input_scale, input_zero_point)
            
        output_details = interpreter.get_output_details()
        output_index = output_details[0]['index']
        output_scale, output_zero_point = output_details[0]['quantization']
            
        input_quantized_list = np.array((input_list / input_scale) + input_zero_point, dtype=np.uint8)
        print(input_quantized_list.shape)
            
        for input_data in input_quantized_list:
            interpreter.set_tensor(input_index, [input_data])
            interpreter.invoke()
                
            output_quantized_data = interpreter.get_tensor(output_index)[0]
            self.predict_list.append(np.argmax(output_quantized_data))
                           
        print("END : " + self.model_path)
            

In [3]:
# --- 분석할 데이터 로딩 ---
# 데이터 구조
# noise_list : 학습용 데이터(이미지 개수, 48, 64, 3)
# noise_label_list : 학습용 라벨 데이터(이미지 개수, 18)

# 학습 데이터 파일명
noise_list_file = "/src/hyebin/esnoise/noise_spec_data.npy"
# 라벨 데이터 파일명
noise_label_list_file = "/src/hyebin/esnoise/noise_label_data.npy"

print("load noise data : ", noise_list_file)
noise_list = np.load(noise_list_file, allow_pickle=True)
print("load complete")

print("load noise label data : ", noise_label_list_file)
noise_label_list = np.load(noise_label_list_file, allow_pickle=True)
print("load complete")

load noise data :  /src/hyebin/esnoise/noise_spec_data.npy
load complete
load noise label data :  /src/hyebin/esnoise/noise_label_data.npy
load complete


In [4]:
# --- 분석 데이터에서 분석 수행용 데이터 일부 추출 ---
# 분석 데이터 설정
# 분석 데이터 중 최초 데이터 위치
base_index = 0
# 분석할 데이터의 개수
analysis_count = 100

# 분석용 데이터와 라벨
analysis_x = noise_list[base_index:base_index + analysis_count]
analysis_y = noise_label_list[base_index:base_index + analysis_count]

In [5]:
# --- 분석 모델 로딩 및 분석 수행 ---
# 소음 분석 모델 디렉토리명
modeldir = "/src/hyebin/model/NOISE-EDGE-SPEC-02.tflite"

analyzer = EsNoiseAnalyzer(modeldir)
analyzer.analysis(analysis_x)

START : /src/hyebin/model/NOISE-EDGE-SPEC-02.tflite
0.007874015718698502 128
(100, 48, 64, 3)
END : /src/hyebin/model/NOISE-EDGE-SPEC-02.tflite


In [6]:
# --- 분석 결과 출력 ---
# 정답 갯수
correct_count = 0
# 오답 갯수
incorrect_count = 0

print("=======================================")
print("index\tpredict\tlabel\tcorrect")
print("---------------------------------------")

for index in range(analysis_count):
    # 예측값 
    predict = analyzer.predict_list[index]
    # 실제값
    label = analysis_y[index]
    
    result = "0"
    
    # 정답일 경우, 정답 카운트 1 증가
    # 오답일 경우, 오답 카운트 1 증가 및 출력시 오른쪽에 X 표시
    if predict == label:
        correct_count = correct_count + 1
    else:
        incorrect_count = incorrect_count + 1
        result = "X"
        message = "%d\t%s\t%s\t%s" %(base_index + index, predict, label, result)

        # 현재 이미지 분석 결과 출력 - 틀린것만
        print(message)

# 최종 결과 출력
print("=====================================")
print("running time:", analyzer.elapsed_time)
print("correct count:", correct_count, "\tcorrect percent:", (correct_count/analysis_count) * 100, "%")
print("incorrect count:", incorrect_count)

index	predict	label	correct
---------------------------------------
9	13	5	X
13	2	1	X
23	16	14	X
30	13	5	X
41	12	7	X
52	6	4	X
62	13	16	X
running time: 0
correct count: 93 	correct percent: 93.0 %
incorrect count: 7
